# Assignmet 3
author: SheltonXiao

## 1 偏差和方差
*请用一个具体的案例解释什么是偏差和方差*

偏差指的是我们用于估计真实函数的模型，它的复杂程度可能小于真实函数，这种不对应会带入误差。而方差指的是，我们用于估计真实函数的数据，是真实函数的一个样本，不同样本带来的模型的差距，称为方差。       
举一个具体案例，比如，我们在进行宝可梦进化后cp值的预测，假设这个真实的函数是$f(x)$，当我们使用线性回归函数（比$f(x)$要简单）来预测cp值，所造成的与真实数据的误差称为偏差；而假设所有的宝可梦有1000个（只是一个假设），而我们每次只拿10个的数据去训练模型来预测，这些训练得到的若干个模型与真实函数间的误差称为方差。

## 2 偏差、方差和误差
*偏差与方差和误差之间的关系*

一般认为，模型的误差=偏差+方差。偏差度量的是单个模型的学习能力，而方差度量的是同一个模型在不同数据集上的稳定性。

## 5 pca降维
*如果使用pca降维前是一个三维的椭球，那么把该图形降维成二维是一个什么样的图形*

pca的主要思想是通过最大投影方差对原始空间进行重构，因此降成二维应该是一个椭圆。

## 8 实践题
*尝试使用sklearn，对一组数据先进行特征的简化（使用三种方式），再使用回归模型，最后使用网格搜索调参，观察三种方法的优劣*

还是用boston房价的数据

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

In [2]:
X = boston.data
y = boston.target
features = boston.feature_names
boston_data = pd.DataFrame(X,columns=features)
boston_data["Price"] = y

方法1：特征筛选

In [3]:
import statsmodels.api as sm 
from statsmodels.formula.api import ols
def forward_select(data,target):
    variate=set(data.columns)  
    variate.remove(target)  
    selected=[]
    current_score,best_new_score=float('inf'),float('inf')  

    while variate:
        aic_with_variate=[]
        for candidate in variate:  
            formula="{}~{}".format(target,"+".join(selected+[candidate]))  
            aic=ols(formula=formula,data=data).fit().aic  
            aic_with_variate.append((aic,candidate))  
        aic_with_variate.sort(reverse=True)  
        best_new_score,best_candidate=aic_with_variate.pop()  
        if current_score>best_new_score:  
            variate.remove(best_candidate)  
            selected.append(best_candidate)  
            current_score=best_new_score  
            print("aic is {},continuing!".format(current_score))  
        else:
            print("for selection over!")
            break
    formula="{}~{}".format(target,"+".join(selected))  #最终的模型式子
    print("final formula is {}".format(formula))
    return(selected)

In [4]:
select = forward_select(data=boston_data,target="Price")

aic is 3286.974956900157,continuing!
aic is 3171.5423142992013,continuing!
aic is 3114.0972674193326,continuing!
aic is 3097.359044862759,continuing!
aic is 3069.438633167217,continuing!
aic is 3057.9390497191152,continuing!
aic is 3048.438382711162,continuing!
aic is 3042.274993098419,continuing!
aic is 3040.154562175143,continuing!
aic is 3032.0687017003256,continuing!
aic is 3021.726387825062,continuing!
for selection over!
final formula is Price~LSTAT+RM+PTRATIO+DIS+NOX+CHAS+B+ZN+CRIM+RAD+TAX


In [23]:
X_1 = np.array(boston_data[select])

方法2：Lasso回归，L1正则化

In [6]:
from sklearn import linear_model
reg_lasso = linear_model.Lasso(alpha = 0.5)
reg_lasso.fit(X,y)

Lasso(alpha=0.5)

取大于等于0.005的特征

In [22]:
X_2 = X[:,np.abs(reg_lasso.coef_)>0.005]

方法3：PCA降维

In [28]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(X)

PCA(n_components=4)

In [29]:
pca.explained_variance_ratio_

array([0.80582318, 0.16305197, 0.02134861, 0.00695699])

In [30]:
X_3 = pca.transform(X)

我们使用SVR模型进行回归

In [31]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import cross_val_score

In [34]:
pipe_svr = Pipeline([("StandardScaler",StandardScaler()),
                     ("svr",SVR())])
param_range = [0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0]
param_grid = [{"svr__C":param_range,"svr__kernel":["linear"]},
              {"svr__C":param_range,"svr__gamma":param_range,
               "svr__kernel":["rbf"]}]

In [36]:
gs_1 = GridSearchCV(estimator=pipe_svr,
                  param_grid = param_grid,
                  scoring = 'r2',
                  cv = 10) 

In [37]:
gs_1 = gs_1.fit(X_1,y)
print("方法1网格搜索最优得分：",gs_1.best_score_)
print("网格搜索最优参数组合：\n",gs_1.best_params_)

方法1网格搜索最优得分： 0.6125173398590678
网格搜索最优参数组合：
 {'svr__C': 1000.0, 'svr__gamma': 0.001, 'svr__kernel': 'rbf'}


In [38]:
gs_2 = GridSearchCV(estimator=pipe_svr,
                  param_grid = param_grid,
                  scoring = 'r2',
                  cv = 10) 

In [39]:
gs_2 = gs_2.fit(X_2,y)
print("方法2网格搜索最优得分：",gs_2.best_score_)
print("网格搜索最优参数组合：\n",gs_2.best_params_)

方法2网格搜索最优得分： 0.6022099687344665
网格搜索最优参数组合：
 {'svr__C': 1000.0, 'svr__gamma': 0.001, 'svr__kernel': 'rbf'}


In [40]:
gs_3 = GridSearchCV(estimator=pipe_svr,
                  param_grid = param_grid,
                  scoring = 'r2',
                  cv = 10) 

In [41]:
gs_3 = gs_3.fit(X_3,y)
print("方法3网格搜索最优得分：",gs_3.best_score_)
print("网格搜索最优参数组合：\n",gs_3.best_params_)

方法3网格搜索最优得分： -0.25615640268448747
网格搜索最优参数组合：
 {'svr__C': 1000.0, 'svr__kernel': 'linear'}
